In [1]:
import json

import boto3
import pandas as pd
import sagemaker
from sagemaker.deserializers import CSVDeserializer
from sagemaker.drift_check_baselines import DriftCheckBaselines
from sagemaker.image_uris import retrieve
from sagemaker.inputs import TrainingInput
from sagemaker.lambda_helper import Lambda
from sagemaker.model_metrics import FileSource, MetricsSource, ModelMetrics
from sagemaker.processing import ProcessingInput, ProcessingOutput, ScriptProcessor
from sagemaker.serializers import CSVSerializer
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.clarify_check_step import (
    ClarifyCheckStep,
    ModelBiasCheckConfig,
    ModelExplainabilityCheckConfig,
)
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.lambda_step import (
    LambdaStep,
)
from sagemaker.workflow.parameters import (
    ParameterBoolean,
    ParameterInteger,
    ParameterString,
)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.steps import CreateModelStep, ProcessingStep, TrainingStep
from sagemaker.xgboost.estimator import XGBoost

/Users/isaacmenchacapanecatl/Documents/hospital_wait_time_prediction/.venv/lib/python3.9/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[04/11/25 10:44:42] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=284438;file:///Users/isaacmenchacapanecatl/Documents/hospital_wait_time_prediction/.venv/lib/python3.9/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=593911;file:///Users/isaacmenchacapanecatl/Documents/hospital_wait_time_prediction/.venv/lib/python3.9/site-packages/botocore/credentials.py#1352\1352]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/isaacmenchacapanecatl/Library/Application Support/sagemaker/config.yaml


## Set up a SageMaker Studio notebook and parameterize the pipeline

In [ ]:
# Instantiate AWS services session and client objects
sess = sagemaker.Session()
write_bucket = sess.default_bucket()
write_prefix = "hospital_wait_time_prediction"

read_bucket = sess.default_bucket()
read_prefix = "hospital_wait_time_prediction/raw"

region = sess.boto_region_name
s3_client = boto3.client("s3", region_name=region)
sm_client = boto3.client("sagemaker", region_name=region)
sm_runtiome_client = boto3.client("sagemaker-runtime")

# Fetch SageMaker execution role
sagemaker_role = sagemaker.get_execution_role()

# Full S3 paths
hospital_data_uri = f"s3://{read_bucket}/{read_prefix}/basesdedatos_50K.xlsx"
output_data_uri = f"s3://{write_bucket}/{write_prefix}/"
scripts_uri = f"s3://{write_bucket}/{write_prefix}/scripts"
estimator_output_uri = f"s3://{write_bucket}/{write_prefix}/training_jobs"
processing_output_uri = f"s3://{write_bucket}/{write_prefix}/processing_jobs"
model_eval_output_uri = f"s3://{write_bucket}/{write_prefix}/model_eval"

# Retrieve training image
training_image = retrieve(framework="xgboost", region=region, version="1.3-1")

In [ ]:
# Set names of pipeline objects
pipeline_name = "HospitalWaitTimePredPipeline"
pipeline_model_name = "hospital-wait-time-pred-pipeline"
model_package_group_name = "hospital-wait-time-pred-model-group"
base_job_name_prefix = "hospital-pred"
endpoint_config_name = f"{pipeline_model_name}-endpoint-config"
endpoint_name = f"{pipeline_model_name}-endpoint"

# Set instance types and counts
process_instance_type = "ml.m4.xlarge"
train_instance_count = 1
train_instance_type = "ml.m4.xlarge"
predictor_instance_count = 1
predictor_instance_type = "ml.m4.xlarge"


In [ ]:
# Set up pipeline input parameters

# Set processing instance type
process_instance_type_param = ParameterString(
    name="ProcessingInstanceType",
    default_value=process_instance_type,
)

# Set training instance type
train_instance_type_param = ParameterString(
    name="TrainingInstanceType",
    default_value=train_instance_type,
)

# Set training instance count
train_instance_count_param = ParameterInteger(
    name="TrainingInstanceCount", default_value=train_instance_count
)

# Set deployment instance type
deploy_instance_type_param = ParameterString(
    name="DeployInstanceType",
    default_value=predictor_instance_type,
)

# Set deployment instance count
deploy_instance_count_param = ParameterInteger(
    name="DeployInstanceCount", default_value=predictor_instance_count
)

# Set model approval param
model_approval_status_param = ParameterString(name="ModelApprovalStatus", default_value="Approved")